In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_code(symbol):
    krx = pd.read_csv('./src/krx_code.csv')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_comment_csv(symbol,page):   
    code = get_code(symbol)
    date_list = [] # 날짜
    comment_list = [] # 댓글
    view_list = [] # 조회수
    good_list = [] # 좋아요
    bad_list = [] # 싫어요
    for i in range(1,page+1):
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n')

                date_list.append(root[1].split()[0].replace('.','-'))
                if len(root) == 14: # 답글
                    comment_list.append('답글:'+root[4])
                    view_list.append(root[10])
                    good_list.append(root[11])
                    bad_list.append(root[12])          
                elif len(root) == 13: # 기본
                    comment_list.append(root[3])
                    view_list.append(root[9])
                    good_list.append(root[10])
                    bad_list.append(root[11])
                else: # 에러
                    comment_list.append('error')
                    view_list.append(0)
                    good_list.append(0)
                    bad_list.append(0)   
            except:
                date_list.append('error')
                comment_list.append('error')
                view_list.append(0)
                good_list.append(0)
                bad_list.append(0)   
        print(f'\r{i}페이지 크롤링 완료.',end='')   
    df = pd.DataFrame()
    df['날짜'] = date_list
    df['댓글'] = comment_list
    df['조회수'] = view_list
    df['좋아요'] = good_list
    df['싫어요'] = bad_list
    df.to_csv(f"./src/네이버종토방댓글_{symbol}_{page}.csv", index=False)
    return df

In [3]:
df = get_comment_csv('NAVER',100)

100페이지 크롤링 완료.

,날짜,댓글,조회수,좋아요,싫어요
0,2022-05-13,폭등이다 폭등,4,0,0
1,2022-05-13,다른 주식 다오르는데 //////////...,12,1,1
2,2022-05-13,부릉부릉,6,0,0
3,2022-05-13,반등은 못하고..,15,0,0
4,2022-05-13,반등장에 5일선 터치도 못한다?,22,1,1
...,...,...,...,...,...
1995,2022-04-28,흠,101,2,1
1996,2022-04-28,네이버 살바엔,146,0,2
1997,2022-04-28,14마넌...,124,1,0
1998,2022-04-28,구글보다는 삼성이 유력하겠지,242,3,1
